In [22]:
# Bag of Words transformation of the partitioned abstracts
# This script uses the CountVectorizer from sklearn to transform the partitioned abstracts into a bag of words representation.

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vect = CountVectorizer()

# Gets the data from Data/partitioned_abstracts.csv and puts it into a pandas dataframe
df = pd.read_csv('Data/Partitioned_Abstracts.csv', sep=',', header=0, encoding='utf-8')

# Add target column to the dataframe based on label
label_mapping = {
    'AI and Supply Chain Management': 0,
    'AI and Finance': 1,
    'AI and Marketing': 2,
    'AI and Economics': 3,
    'AI and Accounting': 4
}
df['target'] = df['Label'].map(label_mapping)


x_train_counts = count_vect.fit_transform(df['Partitioned Abstract'])
df
#x_train_counts.shape

,Title,Year,Authors,Label,Partitioned Abstract,target
0,An analysis to understand the role of machine ...,2022,"Boddu, RSK; Santoki, AA; Khurana, S; Koli, PV;...",AI and Marketing,focusing assumption physical cognitive activit...,2
1,"Gigification, job engagement and satisfaction:...",2022,"Braganza, A; Chen, WF; Canhoto, A; Sap, S",AI and Supply Chain Management,innovative highly efficient artificial intelli...,0
2,Artificial intelligence based decision-making ...,2022,"Lehner, OM; Ittonen, K; Silvola, H; Strm, E; ...",AI and Accounting,trustworthiness using rest's component model a...,4
3,ENHANCING TRUST IN DIGITAL FINANCE: BLOCKCHAIN...,2025,"Luckyardi, S; Fahrezi, M; Soegoto, ES",AI and Finance,rapid advancement financial technology fintech...,1
4,Assessing the Impact of AI Solutions' Ethical ...,2022,"Varzaru, AA",AI and Accounting,contemporary constantly changing business envi...,4
...,...,...,...,...,...,...
738,Beyond Preferences in AI Alignment,2024,"Zhi-Xuan, T; Carroll, M; Franklin, M; Ashton, H",AI and Economics,dominant practice ai alignment assumes 1 prefe...,3
739,Integrating AI in Supply Chain Management: Usi...,2024,"Soares, AL; Gomes, J ; Zimmermann, R; Rhodes, ...",AI and Supply Chain Management,decades collaborative networks community studi...,0
740,It just would not work for me: perceived prefe...,2025,"Sun, LP; Tang, YF; Ma, XY",AI and Economics,rather affective trust effect eliminated unfav...,3
741,Harnessing AI for Sustainable Shipping and Gre...,2024,"Durlik, I; Miller, T; Kostecka, E; Lobodzinska...",AI and Supply Chain Management,maritime industry responsible moving approxima...,0


In [23]:
count_vect.vocabulary_.get(u'algorithm')

543

In [24]:
#TF-IDF transformation

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(x_train_counts)
X_train_tf = tf_transformer.transform(x_train_counts)
X_train_tf.shape


(743, 7810)

In [25]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
X_train_tfidf.shape

(743, 7810)